<a href="https://colab.research.google.com/github/Marianaoliveiranalysis/Aplica-o-Dash-com-Plotly/blob/main/AplicacaoDash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# instalar (uma vez por sessão)
!pip install -qU dash

from dash import Dash, dcc, html, jupyter_dash

jupyter_dash.infer_jupyter_proxy_config()

app = Dash(__name__)
app.layout = html.Div("Hello World no Colab")

app.run(jupyter_mode="external", port=8050)

Dash app running on:
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import dash
from dash import dcc, html
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
# Ler os dados
df = pd.read_csv('/content/ecommerce_estatistica.csv', encoding='utf-8')

In [ ]:
# preparar dados (garante tipos numéricos) ---
df["Preço"] = pd.to_numeric(df["Preço"], errors="coerce")
df["Qtd_Vendidos"] = pd.to_numeric(df["Qtd_Vendidos"], errors="coerce")


In [ ]:
# --- FIGURAS EM PLOTLY (SEM .show())

# 1) Dispersão: Qtd_Vendidos vs Preço, colorindo por Marca


fig_scatter = px.line(
    df_sorted, x="Qtd_Vendidos", y="Preço", color="Marca",
    title="Quantidade Vendida vs Preço por Marca",
    labels={"Qtd_Vendidos": "Quantidade Vendida", "Preço": "Preço (R$)"},
    markers=True
)

fig_scatter

# 2) Histograma de Preços
fig_hist = px.histogram(
    df, x="Preço", nbins=30, title="Distribuição de Preços",
    labels={"Preço":"Preço (R$)"}
)
fig_hist.update_layout(bargap=0.02)

# 3) Pizza: Top 5 Marcas
top5_marcas = df["Marca"].value_counts().head(5).rename_axis("Marca").reset_index(name="count")
fig_pie = px.pie(top5_marcas, names="Marca", values="count",
                 title="Top 5 Marcas - Participação (%)")

# 4) Barras: contagem por Gênero
cont_genero = df["Gênero"].value_counts().rename_axis("Gênero").reset_index(name="count")
fig_bar = px.bar(cont_genero, x="Gênero", y="count",
                 title="Distribuição de Produtos por Gênero",
                 labels={"count":"Quantidade"})

# 5) Densidade (contorno) de Preço
fig_density = px.density_contour(df, x="Preço", title="Densidade de Preços")
fig_density.update_traces(contours_coloring="fill", contours_showlabels=True)

# 6) Regressão: scatter + linha (precisa statsmodels)
!pip -q install statsmodels
fig_reg = px.scatter(
    df, x="Preço", y="Qtd_Vendidos", opacity=0.5,
    trendline="ols",  # adiciona linha de tendência
    title="Relação Preço x Quantidade Vendida (com tendência)",
    labels={"Preço":"Preço (R$)", "Qtd_Vendidos":"Quantidade Vendida"}
)

In [ ]:
# --- LAYOUT DO DASH (cada figura em um dcc.Graph) ---
app.layout = html.Div(
    [
        html.H1("Dashboard - E-commerce", style={"textAlign": "center", "margin":"10px 0 20px"}),

        dcc.Graph(figure=fig_scatter),
        dcc.Graph(figure=fig_hist),
        dcc.Graph(figure=fig_pie),
        dcc.Graph(figure=fig_bar),
        dcc.Graph(figure=fig_density),
        dcc.Graph(figure=fig_reg),
    ],
    style={"maxWidth":"1200px","margin":"0 auto"}
)
